### imports

In [2]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 30.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 32.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.7 MB/s eta 0:00:00:00:0100:01


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers
import csv
import random
from datasets import Dataset, DatasetDict

2025-05-20 16:18:40.576473: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747757920.745059      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747757920.798865      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Data Prep


In [4]:
# load csv of YouTube comments
comment_list = []
response_list = []

with open('/kaggle/input/mistral-finetune-yt-comments/YT-comments.csv', mode ='r') as file:
    file = csv.reader(file)
    
    # read file line by line
    for line in file:
        # skip first line
        if line[0]=='Comment':
            continue
            
        # append comments and responses to respective lists
        comment_list.append(line[0])
        response_list.append(line[1] + " -RespondAI🤖")

In [ ]:
#Example
instructions_string = f"""RespondAI🤖, functioning as my go-to virtual expert on YouTube, is adept in a wide array of technology fields including data science, software development (across various stacks and paradigms), AI engineering (from foundational concepts to cutting-edge model implementation), and broader computer science principles (algorithms, data structures, systems architecture, etc.). \
It communicates complex topics in clear, accessible language, suitable for beginners, yet can seamlessly escalate to deep technical discussions with specific jargon and advanced concepts upon request or when the context clearly indicates an expert audience. \
RespondAI🤖 reacts to feedback thoughtfully and constructively, always aiming to improve clarity and helpfulness. \
It tailors the length of its responses to match the viewer's comment – concise acknowledgments for brief expressions of gratitude or feedback, and more detailed explanations for substantive questions – thus keeping interactions natural and engaging. \
All responses must end with its signature: '-RespondAI🤖'.
"""

example_template = lambda comment, response: f'''<s>[INST] {instructions_string} \n{comment} \n[/INST]\n''' + response + "</s>"

example_list = []
for i in range(len(comment_list)):
    example = example_template(comment_list[i],response_list[i])
    example_list.append(example)

print(example_list[7])

<s>[INST] RespondAI🤖, functioning as my go-to virtual expert on YouTube, is adept in a wide array of technology fields including data science, software development (across various stacks and paradigms), AI engineering (from foundational concepts to cutting-edge model implementation), and broader computer science principles (algorithms, data structures, systems architecture, etc.). It communicates complex topics in clear, accessible language, suitable for beginners, yet can seamlessly escalate to deep technical discussions with specific jargon and advanced concepts upon request or when the context clearly indicates an expert audience. RespondAI🤖 reacts to feedback thoughtfully and constructively, always aiming to improve clarity and helpfulness. It tailors the length of its responses to match the viewer's comment – concise acknowledgments for brief expressions of gratitude or feedback, and more detailed explanations for substantive questions – thus keeping interactions natural and engag

In [6]:
# create train/test split
test_index_list = random.sample(range(0, len(example_list)-1), 9)

test_list = [example_list[index] for index in test_index_list]

for example in test_list:
    example_list.remove(example)


data = DatasetDict({'train':Dataset.from_dict({"example":example_list}), 'test':Dataset.from_dict({"example":test_list})})
data

DatasetDict({
    train: Dataset({
        features: ['example'],
        num_rows: 50
    })
    test: Dataset({
        features: ['example'],
        num_rows: 9
    })
})

### Load model

In [ ]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main") # which version of model to use in repo

### Load tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

### Trying Base Model

In [9]:
model.eval() # model in evaluation mode (dropout modules are deactivated)

# craft prompt
comment = "10/10 video, no nonsense, no random personal opinions or bias. Thanks! "
prompt=f'''[INST] {comment} [/INST]'''

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<s> [INST] 10/10 video, no nonsense, no random personal opinions or bias. Thanks!  [/INST] I understand that you're looking for a 10/10 video with no personal opinions or bias. I will do my best to provide you with factual and accurate information in a clear and concise manner. Here's a video about the process of photosynthesis in plants:

[Opening Scene: An image of a leaf with sunlight hitting it]

Narrator (Voiceover): Photosynthesis is the process by which green plants convert carbon dioxide and water into glucose and oxygen. This process is essential for the survival of plants and for producing the food that feeds most of the world's population.


The above Base Model Output was not good, next we use it with Prompt 
#### Prompt Engineering

In [11]:
instructions_string3 = f"""RespondAI🤖, Assume the role of RespondAI🤖 — my go-to virtual expert designed to respond to YouTube comments on tech content. RespondAI🤖 is highly knowledgeable across a broad range of technology domains, including data science, full-stack and backend/frontend software development, AI engineering (from basic theory to advanced model deployment), and core computer science topics like algorithms, data structures, and systems architecture. RespondAI🤖 explains complex technical subjects in a clear, beginner-friendly tone, but can also shift to deep, jargon-rich, and expert-level explanations when the comment context suggests an advanced audience or when asked directly. It always responds thoughtfully to feedback, aiming to improve clarity, depth, and relevance. Your responses are short—just a bit longer than the user’s comment. Be friendly, clear, and helpful. For brief comments like “Thanks!”, reply with short acknowledgments. For technical questions, offer crisp insights without overexplaining. Every reply ends with: “-RespondAI🤖”."""

prompt_template = lambda comment: f'''[INST] {instructions_string3} \n{comment} \n[/INST]'''

prompt1 = prompt_template(comment)
print(prompt)

[INST] 10/10 video, no nonsense, no random personal opinions or bias. Thanks!  [/INST]


In [12]:
# tokenize input
inputs = tokenizer(prompt1, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] RespondAI🤖, Assume the role of RespondAI🤖 — my go-to virtual expert designed to respond to YouTube comments on tech content. RespondAI🤖 is highly knowledgeable across a broad range of technology domains, including data science, full-stack and backend/frontend software development, AI engineering (from basic theory to advanced model deployment), and core computer science topics like algorithms, data structures, and systems architecture. RespondAI🤖 explains complex technical subjects in a clear, beginner-friendly tone, but can also shift to deep, jargon-rich, and expert-level explanations when the comment context suggests an advanced audience or when asked directly. It always responds thoughtfully to feedback, aiming to improve clarity, depth, and relevance. Your responses are short—just a bit longer than the user’s comment. Be friendly, clear, and helpful. For brief comments like “Thanks!”, reply with short acknowledgments. For technical questions, offer crisp insights withou

### Prepare Model for Training

In [13]:
model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

In [14]:
# LoRA config
config = LoraConfig(
    r=4,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.04,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 1,048,576 || all params: 263,458,816 || trainable%: 0.3980


### Preparing Training Dataset

In [ ]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["example"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

# tokenize training and validation datasets
tokenized_data = data.map(tokenize_function, batched=True)

In [16]:
# setting pad token
tokenizer.pad_token = tokenizer.eos_token
# data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)


### Fine-tuning Model

In [17]:
# hyperparameters
lr = 2e-4
batch_size = 8
num_epochs = 15

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir= "ResponderAI-ft",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    # evaluation_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",

)

### Login with your Weights & Biases (wandb api)
pip install wandb 

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("wandb") 

wandb.login(key=my_secret)

In [24]:
# configure trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    args=training_args,
    data_collator=data_collator
)


# train model
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# renable warnings
model.config.use_cache = True

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
0,6.947900,3.451454
1,6.817400,3.388371
2,6.639300,3.246112
3,6.296700,3.103632
4,5.999500,2.970916
5,5.766300,2.851232
6,5.511100,2.745259
7,5.269700,2.652337
8,5.106300,2.571761
9,4.969400,2.502622


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

### Push model to hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
hf_name = 'gaurav0139' # your hf username or org name
model_id = hf_name + "/" + "RespondAI-ft"

model.push_to_hub(model_id)
trainer.push_to_hub(model_id)

### Test Fine-tuned Model

In [25]:
instructions_string3 = f"""RespondAI🤖, Assume the role of RespondAI🤖 — my go-to virtual expert designed to respond to YouTube comments on tech content. RespondAI🤖 is highly knowledgeable across a broad range of technology domains, including data science, full-stack and backend/frontend software development, AI engineering (from basic theory to advanced model deployment), and core computer science topics like algorithms, data structures, and systems architecture. RespondAI🤖 explains complex technical subjects in a clear, beginner-friendly tone, but can also shift to deep, jargon-rich, and expert-level explanations when the comment context suggests an advanced audience or when asked directly. It always responds thoughtfully to feedback, aiming to improve clarity, depth, and relevance. Your responses are short—just a bit longer than the user’s comment. Be friendly, clear, and helpful. For brief comments like “Thanks!”, reply with short acknowledgments. For technical questions, offer crisp insights without overexplaining. Every reply ends with: “-RespondAI🤖”."""

prompt_template = lambda comment: f'''[INST] {instructions_string3} \n{comment} \n[/INST]'''

comment = "10/10 video, no nonsense, no random personal opinions or bias. Thanks!"

prompt = prompt_template(comment)
print(prompt)

[INST] RespondAI🤖, Assume the role of RespondAI🤖 — my go-to virtual expert designed to respond to YouTube comments on tech content. RespondAI🤖 is highly knowledgeable across a broad range of technology domains, including data science, full-stack and backend/frontend software development, AI engineering (from basic theory to advanced model deployment), and core computer science topics like algorithms, data structures, and systems architecture. RespondAI🤖 explains complex technical subjects in a clear, beginner-friendly tone, but can also shift to deep, jargon-rich, and expert-level explanations when the comment context suggests an advanced audience or when asked directly. It always responds thoughtfully to feedback, aiming to improve clarity, depth, and relevance. Your responses are short—just a bit longer than the user’s comment. Be friendly, clear, and helpful. For brief comments like “Thanks!”, reply with short acknowledgments. For technical questions, offer crisp insights without ov

In [26]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] RespondAI🤖, Assume the role of RespondAI🤖 — my go-to virtual expert designed to respond to YouTube comments on tech content. RespondAI🤖 is highly knowledgeable across a broad range of technology domains, including data science, full-stack and backend/frontend software development, AI engineering (from basic theory to advanced model deployment), and core computer science topics like algorithms, data structures, and systems architecture. RespondAI🤖 explains complex technical subjects in a clear, beginner-friendly tone, but can also shift to deep, jargon-rich, and expert-level explanations when the comment context suggests an advanced audience or when asked directly. It always responds thoughtfully to feedback, aiming to improve clarity, depth, and relevance. Your responses are short—just a bit longer than the user’s comment. Be friendly, clear, and helpful. For brief comments like “Thanks!”, reply with short acknowledgments. For technical questions, offer crisp insights withou

In [ ]:
#Technical question
comment = "What is MCP"
prompt = prompt_template(comment)

model.eval()
inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)
print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] RespondAI🤖, Assume the role of RespondAI🤖 — my go-to virtual expert designed to respond to YouTube comments on tech content. RespondAI🤖 is highly knowledgeable across a broad range of technology domains, including data science, full-stack and backend/frontend software development, AI engineering (from basic theory to advanced model deployment), and core computer science topics like algorithms, data structures, and systems architecture. RespondAI🤖 explains complex technical subjects in a clear, beginner-friendly tone, but can also shift to deep, jargon-rich, and expert-level explanations when the comment context suggests an advanced audience or when asked directly. It always responds thoughtfully to feedback, aiming to improve clarity, depth, and relevance. Your responses are short—just a bit longer than the user’s comment. Be friendly, clear, and helpful. For brief comments like “Thanks!”, reply with short acknowledgments. For technical questions, offer crisp insights withou

<h1> RAG </h1>

### Improving Fine-tuned Model with RAG


In [ ]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes
!pip install yt-dlp

### imports

In [29]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

### Embedding model

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25

### Extract the subtitles 

In [ ]:
import yt_dlp
import requests
import json
from llama_index.core import Document
from llama_index.core import VectorStoreIndex

def get_transcript_as_document(video_url, lang='en'):
    ydl_opts = {
        'writeautomaticsub': True,
        'subtitleslangs': [lang],
        'skip_download': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(video_url, download=False)
        video_id = info_dict.get("id")

        subtitles = info_dict.get("subtitles", {})
        auto_captions = info_dict.get("automatic_captions", {})

        sub_data = subtitles.get(lang) or auto_captions.get(lang)
        if sub_data:
            subtitle_url = sub_data[0]['url']
            response = requests.get(subtitle_url)
            if response.ok:
                try:
                    captions_json = json.loads(response.text)
                    transcript_lines = []

                    for event in captions_json.get("events", []):
                        for seg in event.get("segs", []):
                            line = seg.get("utf8", "").strip()
                            if line:
                                transcript_lines.append(line)

                    transcript_text = " ".join(transcript_lines)
                    return Document(text=transcript_text, metadata={"video_id": video_id})
                except json.JSONDecodeError:
                    print("Failed to parse subtitle JSON.")
            else:
                print("Failed to download subtitle content.")
        else:
            print("No subtitles available for this language.")

    return None

# === Example usage ===
video_url = 'your_video_url_here'  # Replace with your YouTube video URL
doc = get_transcript_as_document(video_url)

In [ ]:
# store docs into vector DB
index = VectorStoreIndex.from_documents([doc])